In [1]:
import gurobipy as grb
import numpy as np
import itertools

In [2]:
S, I, J, K, L, M = 15, 3, 5, 6, 5, 4
plan = [0, 1, 2]
def iterer(*args):
    return itertools.product(*[range(x_) for x_ in args])

In [3]:
model = grb.Model("eurobot")

x = model.addVars(range(S), range(I), range(J), range(K), range(M), name = 'x', vtype=grb.GRB.BINARY)
y = model.addVars(range(S), range(I), range(J), range(L), name = 'y', vtype=grb.GRB.BINARY)

## Unique action per step

In [5]:
model.addConstrs((x.sum(s, '*', '*', '*', '*') == 1 for s in iterer(S)));
model.addConstrs((y.sum(s, i, j, '*') == x.sum(s, i, j, '*', '*') for s, i, j in iterer(S, I, J)));

## Unique cubes and places constraints

In [6]:
model.addConstrs((y.sum('*', i, '*', l) == 1 for i, l in iterer(I, L)));
model.addConstrs((x.sum('*', j, k, '*', '*') <= 1 for j, k in iterer(J, K)));

## Tower height

In [7]:
model.addConstrs((y.sum('*', i, '*', '*') == 5 for i in iterer(I)));

## Bonus plan requirements

In [8]:
pi = model.addVars(range(I), range(6), name = 'pi', vtype=grb.GRB.BINARY)

plan_locations = [[0, 1, 2], [1, 2, 3], [2, 3, 4], [4, 3, 2], [3, 2, 1], [2, 1, 0]]
for i, t in iterer(I, 6):
    for j in range(3):
        model.addConstr(pi[i, t] <= y.sum('*', i, plan[j],  plan_locations[t][j]))

xi = model.addVars(range(I), name='xi', vtype=grb.GRB.BINARY)
model.addConstrs((xi[i] <= pi.sum(i, '*') for i in range(I)))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

## l-Ordering

In [9]:
for s, l1, i in iterer(S,L,I):
    for l2 in range(l1+1,L):
        model.addConstr(grb.quicksum((y.sum(s_,i,'*',l1) for s_ in iterer(s) )) >= y.sum(s,i,'*',l2))

## Ban collisions with other cubes

In [10]:
j_rel = np.zeros((J+1,J+1,2))
j_map = np.array([
                 [-1,-1,-1,-1,-1],
                 [-1,-1, 2,-1,-1],
                 [-1, 3, 4, 1,-1],
                 [-1,-1, 0,-1,-1],
                 [-1,-1,-1,-1,-1]
                 ])

j_on_map = np.zeros((J+1,2))


for i1 in range(j_map.shape[0]):
    for j1 in range(j_map.shape[1]):
        if j_map[i1,j1] >= 0:
            j_on_map[j_map[i1,j1]] = np.array([j1,i1])

for i1 in range(j_map.shape[0]):
    for j1 in range(j_map.shape[1]):
        for i2 in range(j_map.shape[0]):
            for j2 in range(j_map.shape[1]):
                if j_map[i1,j1] >= 0 and j_map[i2,j2] >= 0:
                    if i1 != i2 or j1 != j2:
                        j_rel[j_map[i1,j1],j_map[i2,j2]] = np.array([j2-j1,i2-i1])
                        
# we need just to have free space on 3 'kletka' around the manipulator
def m_manipulator(i,m):
    return (m+i-1)%4

only_non_free_places = [np.array([0,-1]),np.array([-1,0]),np.array([0,1]),np.array([1,0])]

def only_non_free_dif(i,m):
    direction =  m_manipulator(i,m)
    return only_non_free_places[direction]

def our_position(j1):
    return j_on_map[j1]

def list_of_possible_breaks(j1,i,m):
    toreturn = []
    dxy = [np.array([0,1]),np.array([1,0]),np.array([-1,0]),np.array([0,-1])]
    for dd in dxy:
        if (dd != only_non_free_dif(i,m)).any():
            yx = j_on_map[j1] + dd
            if j_map[int(yx[1]),int(yx[0])] >= 0:
                toreturn.append(j_map[int(yx[1]),int(yx[0])])
    return toreturn

list_of_possible_breaks(2, 2, 3)

[4]

In [248]:
def debug_possible_breaks(j1, i, m):
    return [0] if j1 == 4 else []

In [11]:
for s, i, j, k, m in iterer(S, I, J, K, M):
    cubes_to_peak = list_of_possible_breaks(j, i, m)
#     print("%d %d %d"%(i,j,m) + ' ' + str(cubes_to_peak))
    left_gen = (x.sum(s_prev, '*', cube, k, '*') for s_prev, cube in itertools.product(range(s), cubes_to_peak))
#         expr = grb.LinExpr()
#         for s_prev, cube in itertools.product(range(s), cubes_to_peak):
#             expr += x.sum(s_prev, '*', cube, k, '*', '*')
    if s> 0 and len(cubes_to_peak) > 0:    
        model.addConstr(grb.quicksum(left_gen) >= x.sum(s, i, j, k, m))

## Time

In [15]:
v = 20
t_p = 2 # time of picking
t_r = 1 # time of rotation on pi / 2
t_m = 2 # time of moving on 6 cm

picking_points = np.zeros((5, 4, 3, 3))
cubes_coords = np.array([[0, -1], [1, 0], [0, 1], [-1, 0], [0, 0]])
l1 = np.array([[[1, 0], [0, 1], [-1, 0]],
               [[0, 1], [-1, 0], [0, -1]],
               [[-1, 0], [0, -1], [1, 0]],
               [[0, -1], [1, 0], [0, 1]]])

picking_points[..., :2] = cubes_coords[:, np.newaxis, np.newaxis, :] + l1[np.newaxis, :, :, :]
picking_points[..., 2] = np.array(range(4))[np.newaxis, :, np.newaxis]

points_2 = np.reshape(picking_points, (60, 3))
small_time_2 = t_m * np.abs(points_2[:, np.newaxis, 0] - points_2[np.newaxis, :, 0]) + \
             t_m * np.abs(points_2[:, np.newaxis, 1] - points_2[np.newaxis, :, 1]) + \
             t_r * ((points_2[:, np.newaxis, 2] - points_2[np.newaxis, :, 1]) % 4)
        
small_time = np.where(small_time_2 == 0, 0, small_time_2 + t_p)
heap_points = np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]])
all_picking_points = np.reshape(points_2[np.newaxis, :, :2] * 6 + heap_points[:, np.newaxis, :2], (360, 2))
times = np.sum((all_picking_points[:, np.newaxis] - all_picking_points[np.newaxis, :]) ** 2, axis=2) ** 0.5 / v + t_p
for i in range(6):
    times[i * 60: (i + 1) * 60, i * 60: (i + 1) * 60] = small_time
times = times.reshape(6, 5, 4, 3, 6, 5, 4, 3)

In [16]:
%%time
gen = (x.sum(s, i_1, j_1, k_1, m_1) * x.sum(s + 1, i_2, j_2, k_2, m_2) * \
       times[k_1, j_1, m_1, i_1, k_2, j_2, m_2, i_2] for s, i_1, j_1, k_1, m_1, i_2, j_2, k_2, m_2 \
       in iterer(S - 1, I, J, K, M, I, J, K, M))
T = 1000
model.addConstr(grb.quicksum(gen) <= T, 'time');

CPU times: user 3min 29s, sys: 344 ms, total: 3min 30s
Wall time: 3min 30s


## Objective

In [12]:
obj = xi.sum('*')
model.setObjective(obj, grb.GRB.MAXIMIZE)

## Optimization 

In [13]:
model.update()
model.optimize()

Optimize a model with 5067 rows, 6546 columns and 598737 nonzeros
Variable types: 0 continuous, 6546 integer (6546 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 3414 rows and 3582 columns
Presolve time: 1.36s
Presolved: 1653 rows, 2964 columns, 90921 nonzeros
Variable types: 0 continuous, 2964 integer (2964 binary)
Found heuristic solution: objective 1.0000000
Found heuristic solution: objective 3.0000000

Explored 0 nodes (0 simplex iterations) in 1.76 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 3 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [14]:
model.printAttr('x')


    Variable            x 
-------------------------
x[0,0,0,0,0]            1 
x[1,0,1,0,2]            1 
x[2,0,2,1,3]            1 
x[3,0,3,1,0]            1 
x[4,1,0,2,0]            1 
x[5,1,1,2,1]            1 
x[6,1,2,3,2]            1 
x[7,1,3,3,3]            1 
x[8,2,0,4,3]            1 
x[9,2,1,4,0]            1 
x[10,2,2,5,1]            1 
x[11,2,3,5,2]            1 
x[12,2,4,5,3]            1 
x[13,1,4,5,3]            1 
x[14,0,4,5,3]            1 
  y[0,0,0,0]            1 
  y[1,0,1,1]            1 
  y[2,0,2,2]            1 
  y[3,0,3,3]            1 
  y[4,1,0,0]            1 
  y[5,1,1,1]            1 
  y[6,1,2,2]            1 
  y[7,1,3,3]            1 
  y[8,2,0,0]            1 
  y[9,2,1,1]            1 
 y[10,2,2,2]            1 
 y[11,2,3,3]            1 
 y[12,2,4,4]            1 
 y[13,1,4,4]            1 
 y[14,0,4,4]            1 
     pi[0,0]            1 
     pi[1,0]            1 
     pi[2,0]            1 
       xi[0]            1 
       xi[1]           